In [ ]:
def prepare_issue_plot_data(df, issue_df, pid_meas, pid_setpoint, title, color, label, tolerance=0.05):
    # Convert 'time' to datetime and set as index
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)

    # Global x-values (time axis)
    x_values = df.index.to_series().dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

    response = {
        "title": title,
        "xlabel": "Time",
        "ylabel": "Value",
        "x": x_values,  # Shared x-axis
        "series": []
    }

    # Measured Output
    response["series"].append({
        "name": "Measured Output",
        "y": df[pid_meas].tolist(),
        "type": "line",
        "color": "blue",
        "style": "dashed"
    })

    # Setpoint
    response["series"].append({
        "name": "Setpoint",
        "y": df[pid_setpoint].tolist(),
        "type": "line",
        "color": "green",
        "style": "solid"
    })

    # Acceptable Range (for PID1)
    if "PID_1" in pid_setpoint:
        upper_limit = (df[pid_setpoint] * (1 + tolerance)).tolist()
        lower_limit = (df[pid_setpoint] * (1 - tolerance)).tolist()

        response["series"].append({
            "name": "Upper Acceptable Limit",
            "y": upper_limit,
            "type": "line",
            "color": "gray",
            "style": "dotted"
        })
        response["series"].append({
            "name": "Lower Acceptable Limit",
            "y": lower_limit,
            "type": "line",
            "color": "gray",
            "style": "dotted"
        })

    # Issue Points
    if not issue_df.empty:
        # Ensure the issue_df index is datetime and formatted
        issue_df['time'] = pd.to_datetime(issue_df['time'])
        issue_df.set_index('time', inplace=True)
        issue_df = issue_df.loc[df.index]  # Align to main df's timeline if needed

        response["series"].append({
            "name": label,
            "x": issue_df.index.strftime('%Y-%m-%d %H:%M:%S').tolist(),  # still custom x here
            "y": issue_df[pid_meas].tolist(),
            "type": "scatter",
            "color": color,
            "marker": "x"
        })

    return response